# Librairies

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
import numpy as np
from torch.nn.utils.rnn import pad_sequence
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

### Import les modules fait maison

In [ ]:
import os
import sys
src = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(src)
import src.video.video_extract as ve
import src.video.video_model as vm
import src.load_data as load_data

### Chargement du dataset (csv)

In [ ]:
transcr_path = '../paco-cheese/transcr'
data = load_data.load_all_ipus(folder_path=transcr_path, load_words=True) #fonction donnée par le prof

### Réduire le dataset pour réaliser des test

In [ ]:
df = pd.DataFrame(data)
df_save=df.copy()

In [ ]:
#pour avoir juste une video et raccourcir df (test)
df=df_save[df_save['dyad']=='transcr\MAPC'][0:2] #pour prendre peu de temps sinon vraiment trop lourd et long
df.shape

### Fonction `find_video_file` permettant de synchroniser les vidéos à leur ligne dans le dataframe

In [ ]:
# video_files_path = 'paco-cheese/video/video/'
#trouver les path des videos
def find_video_file(dyad, first_speaker):
    dyad = dyad.split('\\')[1]
    if isinstance(first_speaker, float):
        first_speaker = "NA"

    second_speaker = dyad.replace(first_speaker, "")

    subdirs = ['cheese', 'paco']
    for subdir in subdirs:
        media_files_path = f'../paco-cheese/video/video/{subdir}/'
        for file_name in os.listdir(media_files_path):
            if first_speaker in file_name and second_speaker in file_name:
                return os.path.join(media_files_path, file_name)

    return None
for index, row in df.iterrows():
    first_speaker = str(row['speaker']) if not pd.isna(row['speaker']) else "NA"
    video_path = find_video_file(row['dyad'], first_speaker)

    print(video_path)

### Utilisation du module video_extract pour extraire les features des differentes vidéos

### Plus d'explications sur le module d'extraction :

Le code ci-dessus implémente un processus pour extraire, traiter et analyser des caractéristiques à partir de vidéos :

1. **Extraction des segments vidéo** : La fonction `extract_video_segments` utilise OpenCV pour extraire des frames d'une vidéo entre des timestamps spécifiés (de `start_ms` à `end_ms`). Elle utilise `cv2.VideoCapture` pour lire la vidéo et extrait les frames pertinentes en fonction du framerate de la vidéo (`fps`).

2. **Prétraitement des frames** : `preprocess_frame` redimensionne chaque frame à une taille fixe (224x224), nécessaire pour des modèles de réseaux de neurones comme VGG16 (qu'on utilisait précédemment, on a décider de garder cette taille).

3. **Extraction de caractéristiques avec FeatureExtractor** :
    - Initialise un modèle de détection de visages (`face_net`) et un modèle de landmarks faciaux (`landmark_model`).
    - La méthode `extract` détecte les visages dans une frame, puis extrait et dessine les landmarks faciaux. Pour chaque détection réussie, elle collecte les keypoints des landmarks faciaux.

4. **Fusion des caractéristiques** : La fonction `fuse_features` permet de combiner les caractéristiques extraites de différentes manières selon les besoins spécifiques de l'application.

5. **Extraction et traitement des caractéristiques vidéo** :
    - Boucle sur chaque enregistrement dans un DataFrame, extrait le chemin du fichier vidéo, et pour chaque vidéo, extrait les frames, les prétraite, et extrait les caractéristiques avec `FeatureExtractor`.
    - Les caractéristiques de la vidéo sont ensuite fusionnées et stockées dans `X_video`.

Ce processus est conçu pour transformer des segments vidéo en un ensemble de caractéristiques prêtes à être analysées ou utilisées pour l'entraînement de modèles de machine learning.


In [ ]:
feature_extractor = ve.FeatureExtractor()

X_video = []

# count=0
for index, row in df.iterrows():
    # count=count+1
    # if count==5:
    #     break;
    first_speaker = str(row['speaker']) if not pd.isna(row['speaker']) else "NA"
    video_file_path = find_video_file(row['dyad'], first_speaker)
    #video_file_path="test_2.mp4"
    if video_file_path:
        print(f"Traitement du fichier vidéo : {video_file_path}")
        start_ms = int(row['start_ipu'] * 1000)
        end_ms = int(row['stop_ipu'] * 1000)
        frames = ve.extract_video_segments(video_file_path, start_ms, end_ms)  # Using ve.extract_video_segments

        if not frames:
            print("Aucun cadre extrait du fichier vidéo.")
            X_video.append(np.zeros((512,)))
            continue
        for frame in frames:
            plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            plt.show()

        print(f"Nombre de cadres extraits : {len(frames)}")
        features = []
        for frame in frames:
            frame_preprocessed = ve.preprocess_frame(frame)  # Using ve.preprocess_frame
            keypoints = feature_extractor.extract(frame_preprocessed)
            features.extend(keypoints)  # Concaténation des keypoints de tous les cadres
        video_features = ve.fuse_features(features)  # Using ve.fuse_features
        X_video.append(video_features)
        print(f"Caractéristiques vidéo extraites : {len(video_features)} points détectés.")
    else:
        print(f"Aucun chemin de fichier vidéo trouvé pour {row['dyad']} et {first_speaker}")
        X_video.append(np.zeros((512,)))


# Modèle


### Plus d'informations sur le module dataset et modele :

### Implémentation d'un Modèle LSTM pour l'Analyse Vidéo

Notre projet a également exploré l'intégration de l'analyse vidéo. Bien que notre attention principale soit orientée vers l'audio et le texte, nous avons développé un modèle LSTM basique pour démontrer notre capacité à traiter et analyser les caractéristiques extraites des vidéos.

#### Préparation des Données
- **Padding des séquences vidéo** : Nous avons normalisé la longueur des séquences vidéo en utilisant `pad_sequence` pour que toutes aient la même taille, ce qui est une étape essentielle pour le traitement par LSTM.
- **Aplatissage des Caractéristiques** : Les caractéristiques vidéo extraites ont été remodelées pour s'adapter à l'entrée 3D attendue par le LSTM (batch, seq_len, features).

#### Modèle LSTM
- **Conception** : Le modèle `LSTMClassifier` est une architecture LSTM, avec la possibilité d'ajuster le nombre de couches via `num_layers`. Cela permet de moduler la complexité du modèle en fonction de la nature des données.
- **Optimisation** : Le modèle utilise une fonction de perte BCE et un optimiseur Adam, des choix courants pour les tâches de classification binaire.


<span style="color:red">Par manque de temps, nous avons décidé de nous concentrer sur l'audio et le texte car l'entraînement et l'extraction de features avec des vidéos peut s'avérer très long et nos ordinateurs peu adaptés.</span>



In [ ]:
# Padding de X_video pour que toutes les séquences aient la même longueur
padded_X_video = pad_sequence([torch.tensor(sequence) for sequence in X_video], batch_first=True, padding_value=0)

# Aplatir les deux dernières dimensions pour obtenir une entrée 3D et être dapté au LSTM (batch, seq_len, features)
padded_X_video = padded_X_video.view(padded_X_video.shape[0], padded_X_video.shape[1], -1)

# Conversion de df['turn_after'] en tensor
y_tensor = torch.tensor(df['turn_after'].values)

X_train, X_test, y_train, y_test = train_test_split(padded_X_video, y_tensor, test_size=0.2, random_state=42)

# Créer les data loaders
train_data = DataLoader(list(zip(X_train, y_train)), batch_size=2, shuffle=True)
test_data = DataLoader(list(zip(X_test, y_test)), batch_size=2, shuffle=True)

train_data = DataLoader(list(zip(X_train, y_train)), batch_size=2, shuffle=True)
test_data = DataLoader(list(zip(X_test, y_test)), batch_size=2, shuffle=True)

# Parametres
embedding_dim = padded_X_video.shape[2]
hidden_dim = 64
label_size = 1
num_layers = 5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Creer le model (sur CPU si dispo ou voulu)
model = vm.LSTMClassifier(embedding_dim, hidden_dim, label_size, num_layers=num_layers).to(device)

# Loss et optimizer
loss_function = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Entrainement
vm.train_model(model, train_data, loss_function, optimizer, num_epochs=10, device=device)

# Evaluation
test_labels, test_predictions = vm.evaluate_model(model, test_data, device=device)

# Ici on applique un seuil pour pouvoir mieux prédire (fait office de poids sur la classe positive, ça change un peu des autres modèles)
test_predictions = [1 if prob > 0.5 else 0 for prob in test_predictions]
print(classification_report(test_labels, test_predictions))
